In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import scipy as scp
import math

In [3]:
# EXAMPLE WITH 4GEN
def COPT4():
    # initial values
    Cgens = [200, 100, 200, 500]
    Ugens = [0.05, 0.03, 0.04, 0.06]

    COPT = np.zeros([np.int_(sum(Cgens)/100+1),3])
    COPT[:,0] = range(0,np.int_(sum(Cgens)+100),100)
    Agens = np.subtract([1,1,1,1],Ugens)
    A_allgens = math.prod(Agens)


    ## 0th-order
    COPT[0,1] = A_allgens


    ## 1st-order
    for i in range(4):
        Pstate = Ugens[i]*A_allgens/Agens[i]
        COPT[np.int_(Cgens[i]/100),1] = COPT[np.int_(Cgens[i]/100),2] + Pstate


    ## 2nd-order
    for i in range(3):
        for j in range(i,4):
            Pstate = Ugens[i]*Ugens[j]*A_allgens/(Agens[i]*Agens[j])
            COPT[np.int_((Cgens[i] + Cgens[j])/100),1] = COPT[np.int_((Cgens[i] + Cgens[j])/100),2] + Pstate


    ## 3rd-order
    for i in range(2):
        for j in range(i,3):
            for k in range(i,4):
                Pstate = Ugens[i]*Ugens[j]*Ugens[k]*A_allgens/(Agens[i]*Agens[j]*Agens[k])
                COPT[np.int_((Cgens[i] + Cgens[j] + Cgens[k])/100),1] = COPT[np.int_((Cgens[i] + Cgens[j] + Cgens[k])/100),1] + Pstate


    ## 4th-order
    Pstate = np.prod(Ugens)
    COPT[np.int_(sum(Cgens)/100),1] = COPT[np.int_(sum(Cgens)/100),1] + Pstate


    ## results

    COPT[0,2] = 1

    for i in range(np.int_(sum(Cgens)/100+1)):
        COPT[i,2] = 1-sum(COPT[0:i,1])
    return COPT

In [4]:
Cgens = [200, 100, 200, 500]
Ugens = [0.05, 0.03, 0.04, 0.06]
load = [500, 1000, 400, 600, 900, 100, 300] # for every hour for 7 hours
# Create a dataframe wind_pdf with the probability distribution of the wind power, 25% to generate 100, 25% to generate 200, 15% to generate 300, 35% to generate 1000.
wind_pdf = pd.DataFrame(
    {
        "Capacity": [100, 200, 300, 1000],
        "Probability": [0.25, 0.25, 0.15, 0.35],
    }
)
total_cap = np.sum(Cgens)
LOLP = 0.03655 
LOLE = 0.03655 * 8760
df_COPT4 = pd.DataFrame(COPT4(), columns=['Capacity', 'Probability', 'Cumulative Probability'])
df_COPT4

,Capacity,Probability,Cumulative Probability
0,0.0,0.831562,1.000000
1,100.0,0.025718,0.168438
2,200.0,0.000795,0.142720
3,300.0,0.001096,0.141925
4,400.0,0.001552,0.140828
5,500.0,0.053378,0.139277
6,600.0,0.002031,0.085898
7,700.0,0.002262,0.083867
8,800.0,0.000155,0.081605
9,900.0,0.000263,0.081450
